In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.integrate import cumtrapz
import tensorflow as tf
from tensorflow.keras import optimizers
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Bidirectional,Dropout

from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling1D
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from statistics import mean, stdev
import warnings
warnings.filterwarnings(action='ignore') 

In [6]:
df=pd.read_csv('./new_all.csv')

In [7]:
df

,ID,e_type,date_time,H_ele,APT,size,PersonCount,month,weekday,time,weekend
0,0,전력총합,2012-03-01 0,0.146,1,20평~29평,2,3,목,0,0
1,0,전력총합,2012-03-01 1,0.146,1,20평~29평,2,3,목,1,0
2,0,전력총합,2012-03-01 2,0.147,1,20평~29평,2,3,목,2,0
3,0,전력총합,2012-03-01 3,0.156,1,20평~29평,2,3,목,3,0
4,0,전력총합,2012-03-01 4,0.154,1,20평~29평,2,3,목,4,0
...,...,...,...,...,...,...,...,...,...,...,...
3837475,255,모니터,2012-08-31 19,0.060,0,20평~29평,2,8,금,19,0
3837476,255,모니터,2012-08-31 20,0.060,0,20평~29평,2,8,금,20,0
3837477,255,모니터,2012-08-31 21,0.060,0,20평~29평,2,8,금,21,0
3837478,255,모니터,2012-08-31 22,0.060,0,20평~29평,2,8,금,22,0


In [46]:
df['e_type'].value_counts()

전력총합       612792
TV         538416
데스크탑       452928
전기밥솥       420792
세탁기        358920
냉장고        260928
김치냉장고      250704
전자렌지       181896
선풍기        157176
정수기         78264
전기장판        75744
에어컨         56808
노트북         52488
청소기         43728
커피포트        25824
보일러         25032
공기청정기       22704
드라이기        21960
전기히터        21000
냉온수기        20784
모니터         19320
충전기         18672
비데기         17688
다리미         14760
스탠드         11904
어항          11832
공유기         11688
믹서기          8808
수족관          4944
오디오          4944
게임기          4128
가스렌지         3864
가습기          3432
프린터          3096
화장대          2832
런닝머신         2592
토스트기         2520
미싱기          2136
감시카메라        1824
제본기          1272
제습기          1056
커피머신         1008
냉풍기(큰방)       984
전기포트          936
빔프로젝트         888
약탕기           816
전기맛사지기        408
오븐            192
셋톱박스           48
Name: e_type, dtype: int64

In [8]:
df[df.e_type=='약탕기']

,ID,e_type,date_time,H_ele,APT,size,PersonCount,month,weekday,time,weekend
271920,160,약탕기,2012-03-08 0,0.061,0,20평~29평,2,3,목,0,0
271921,160,약탕기,2012-03-08 1,0.061,0,20평~29평,2,3,목,1,0
271922,160,약탕기,2012-03-08 2,0.061,0,20평~29평,2,3,목,2,0
271923,160,약탕기,2012-03-08 3,0.061,0,20평~29평,2,3,목,3,0
271924,160,약탕기,2012-03-08 4,0.061,0,20평~29평,2,3,목,4,0
...,...,...,...,...,...,...,...,...,...,...,...
3302251,160,약탕기,2012-01-13 19,0.148,0,20평~29평,2,1,금,19,0
3302252,160,약탕기,2012-01-13 20,0.156,0,20평~29평,2,1,금,20,0
3302253,160,약탕기,2012-01-13 21,0.150,0,20평~29평,2,1,금,21,0
3302254,160,약탕기,2012-01-13 22,0.156,0,20평~29평,2,1,금,22,0


In [47]:
# e_type 종류는 49개, 총합을 제외한 나머지 48종류
new

,ID,e_type,date_time,H_ele,APT,size,PersonCount,month,weekday,time,weekend
0,0,전력총합,2012-03-01 0,0.146,1,20평~29평,2,3,목,0,0
1,0,전력총합,2012-03-01 1,0.146,1,20평~29평,2,3,목,1,0
2,0,전력총합,2012-03-01 2,0.147,1,20평~29평,2,3,목,2,0
3,0,전력총합,2012-03-01 3,0.156,1,20평~29평,2,3,목,3,0
4,0,전력총합,2012-03-01 4,0.154,1,20평~29평,2,3,목,4,0
...,...,...,...,...,...,...,...,...,...,...,...
3835051,240,전력총합,2012-08-27 19,0.407,0,20평~29평,3,8,월,19,0
3835052,240,전력총합,2012-08-27 20,0.352,0,20평~29평,3,8,월,20,0
3835053,240,전력총합,2012-08-27 21,0.324,0,20평~29평,3,8,월,21,0
3835054,240,전력총합,2012-08-27 22,0.319,0,20평~29평,3,8,월,22,0


In [43]:
new=df[df['e_type']=='전력총합']
new[new['ID']==5]

,ID,e_type,date_time,H_ele,APT,size,PersonCount,month,weekday,time,weekend
1194960,5,전력총합,2012-06-01 0,0.232,1,20평~29평,4,6,금,0,0
1194961,5,전력총합,2012-06-01 1,0.224,1,20평~29평,4,6,금,1,0
1194962,5,전력총합,2012-06-01 2,0.224,1,20평~29평,4,6,금,2,0
1194963,5,전력총합,2012-06-01 3,0.224,1,20평~29평,4,6,금,3,0
1194964,5,전력총합,2012-06-01 4,0.231,1,20평~29평,4,6,금,4,0
...,...,...,...,...,...,...,...,...,...,...,...
3335155,5,전력총합,2012-08-31 19,0.178,1,20평~29평,4,8,금,19,0
3335156,5,전력총합,2012-08-31 20,0.179,1,20평~29평,4,8,금,20,0
3335157,5,전력총합,2012-08-31 21,0.176,1,20평~29평,4,8,금,21,0
3335158,5,전력총합,2012-08-31 22,0.164,1,20평~29평,4,8,금,22,0


In [48]:
gasb=df[df['e_type']=='가습기']
gasb

,ID,e_type,date_time,H_ele,APT,size,PersonCount,month,weekday,time,weekend
13440,5,가습기,2012-03-01 0,0.149,1,20평~29평,4,3,목,0,0
13441,5,가습기,2012-03-01 1,0.177,1,20평~29평,4,3,목,1,0
13442,5,가습기,2012-03-01 2,0.131,1,20평~29평,4,3,목,2,0
13443,5,가습기,2012-03-01 3,0.132,1,20평~29평,4,3,목,3,0
13444,5,가습기,2012-03-01 4,0.167,1,20평~29평,4,3,목,4,0
...,...,...,...,...,...,...,...,...,...,...,...
3112291,5,가습기,2012-01-31 19,0.060,1,20평~29평,4,1,화,19,0
3112292,5,가습기,2012-01-31 20,0.061,1,20평~29평,4,1,화,20,0
3112293,5,가습기,2012-01-31 21,0.064,1,20평~29평,4,1,화,21,0
3112294,5,가습기,2012-01-31 22,0.065,1,20평~29평,4,1,화,22,0


In [36]:
gasb[gasb['e_type']=='가습기'].describe(percentiles=[.10,.20,.30,.40,.60])

,ID,H_ele,APT,PersonCount,month,time,weekend
count,3432.000000,3432.000000,3432.0,3432.0,3432.000000,3432.000000,3432.000000
mean,6.706294,0.080529,1.0,4.0,4.202797,11.500000,0.279720
std,20.335799,0.038967,0.0,0.0,3.346625,6.923195,0.448927
min,5.000000,0.060000,1.0,4.0,1.000000,0.000000,0.000000
10%,5.000000,0.060000,1.0,4.0,1.000000,2.000000,0.000000
20%,5.000000,0.060000,1.0,4.0,2.000000,4.000000,0.000000
30%,5.000000,0.060000,1.0,4.0,2.000000,7.000000,0.000000
40%,5.000000,0.060000,1.0,4.0,3.000000,9.000000,0.000000
50%,5.000000,0.060000,1.0,4.0,3.000000,11.500000,0.000000
60%,5.000000,0.061000,1.0,4.0,4.000000,14.000000,0.000000


In [14]:
onoff_gasb = []

for i in gasb['H_ele']:
    if i > 0.0605:
        onoff_gasb.append(1)
    else:
        onoff_gasb.append(0)

In [41]:
gasb['onoff'] = onoff_gasb

In [42]:
gasb

,ID,e_type,date_time,H_ele,APT,size,PersonCount,month,weekday,time,weekend,onoff
13440,5,가습기,2012-03-01 0,0.149,1,20평~29평,4,3,목,0,0,1
13441,5,가습기,2012-03-01 1,0.177,1,20평~29평,4,3,목,1,0,1
13442,5,가습기,2012-03-01 2,0.131,1,20평~29평,4,3,목,2,0,1
13443,5,가습기,2012-03-01 3,0.132,1,20평~29평,4,3,목,3,0,1
13444,5,가습기,2012-03-01 4,0.167,1,20평~29평,4,3,목,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3112291,5,가습기,2012-01-31 19,0.060,1,20평~29평,4,1,화,19,0,0
3112292,5,가습기,2012-01-31 20,0.061,1,20평~29평,4,1,화,20,0,1
3112293,5,가습기,2012-01-31 21,0.064,1,20평~29평,4,1,화,21,0,1
3112294,5,가습기,2012-01-31 22,0.065,1,20평~29평,4,1,화,22,0,1


In [ ]:
# 전체 전력총합에는 있으나 개별 데이터 수집에는 없는 데이터가 있음
# 즉, 로우 수가 안 맞음.

In [ ]:
feature = new.iloc[:,3:11]
label = new[['onoff']]